In [1]:
import os

In [2]:
%pwd

'd:\\PYTHON\\AI-ML-DS\\DS\\Kidney Classification Model\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\PYTHON\\AI-ML-DS\\DS\\Kidney Classification Model'

In [5]:
from dotenv import load_dotenv         
load_dotenv()  

True

In [6]:
import tensorflow as tf


In [7]:
model = tf.keras.models.load_model("artifacts/training/trained_model.keras")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_to_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from Kidney_Disease_Classsification.constants import *
from Kidney_Disease_Classsification.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_to_model="artifacts/training/trained_model.keras",
            training_data="artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            all_params=self.params,
            mlflow_uri="https://dagshub.com/PSeshpavan/Kidney-Classification.mlflow",
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [24]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    def _valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )
        
        dataflow_kwargs = dict(
           target_size = self.config.params_image_size[:-1],
           batch_size = self.config.params_batch_size,
           interpolation = "bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self._valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        ) 
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    
    def evaluate(self):
        self.model = self.load_model(self.config.path_to_model)
        self._valid_generator()
        self.score = self.model.evaluate(self._valid_generator)
        self.save_scores()
        
        
    def save_scores(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        
        
    def log_to_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}                
            )
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="Kidney_Disease_Classsification USING VGG16 Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [25]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluate()
    evaluation.log_to_mlflow()
except Exception as e:
    raise e

[2025-05-26 15:23:23,874: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-26 15:23:23,875: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-26 15:23:23,876: INFO: common: created directory at: artifacts]
Found 3732 images belonging to 4 classes.


c:\Users\pulla\.conda\envs\kidney\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


234/234 ━━━━━━━━━━━━━━━━━━━━ 131s 560ms/step - accuracy: 0.5506 - loss: 4.3593
[2025-05-26 15:25:37,897: INFO: common: json file saved at: scores.json]


2025/05/26 15:25:38 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/05/26 15:26:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[2025-05-26 15:27:08,122: WARNING: connectionpool: Retrying (Retry(total=6, connect=7, read=7, redirect=7, status=7)) after connection broken by 'SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2427)')': /PSeshpavan/Kidney-Classification.mlflow/api/2.0/mlflow-artifacts/artifacts/24c6985b9fb34dada9516eafe5f7e7ed/ced68ac639f04cf4b0911bdfe0586731/artifacts/model/data/model.keras]


Successfully registered model 'Kidney_Disease_Classsification USING VGG16 Model'.
2025/05/26 15:27:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Kidney_Disease_Classsification USING VGG16 Model, version 1
Created version '1' of model 'Kidney_Disease_Classsification USING VGG16 Model'.


🏃 View run monumental-duck-405 at: https://dagshub.com/PSeshpavan/Kidney-Classification.mlflow/#/experiments/0/runs/ced68ac639f04cf4b0911bdfe0586731
🧪 View experiment at: https://dagshub.com/PSeshpavan/Kidney-Classification.mlflow/#/experiments/0
